In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp
import sys

import sys
sys.path.append("..") 
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.utils import (calc_masks, create_seed_features, load_data,
                          num_layers_dict, prepare_data, random_unlabel)


/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def approx_chunk(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [10]:
closed_accuracies = np.zeros((10)) 
iter_accuracies = np.zeros((10,10)) 
val_accuracies = np.zeros((10,10)) 
for p in [0.99]:
#     for i,data in enumerate(['flip_cora','flip_dblp','flip_flickr','flip_imdb','flip_industry','linqs_citeseer','linqs_cora','linqs_pubmed']):
    for i,data in enumerate(['flip_cora']):
        print(data)
        if data[:4] == 'flip':
            model = 'att'
        else:
            model = 'edge'

        for seed in range(10):
            true_labels, edge_features, graph \
            = load_data(data,model=model)

            labeled_indices, unlabeled_indices = \
                random_unlabel(true_labels,p,
                               seed=seed)
            num_nodes, num_classes = true_labels.shape

            labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices)

            for j in range(1,11):
                final_accs = []

                labeled_indices_copy = copy.copy(labeled_indices)
                random.seed(seed)
                shuffle(labeled_indices_copy)
                cv_held_out_indices_list = approx_chunk(labeled_indices_copy, 5)


                for k, cv_held_out_indices in enumerate(cv_held_out_indices_list):

                    cv_labeled_indices = [index for index in labeled_indices if index not in cv_held_out_indices]
                    cv_unlabeled_indices = np.delete(np.arange(true_labels.shape[0]),cv_labeled_indices)
                    cv_labels, cv_is_labeled = calc_masks(true_labels, cv_labeled_indices, cv_unlabeled_indices)
                    lp = LP()
                    index = np.hstack([cv_held_out_indices,cv_unlabeled_indices])
                    unlabeled_pred = lp.iter_sp(cv_labels,
                                         graph,
                                         cv_is_labeled,
                                         10 * j,
                                         index)

                    y_pred = np.argmax(unlabeled_pred[:len(cv_held_out_indices)],axis=1)
                    y_true = np.argmax(true_labels[cv_held_out_indices],axis=1)
                    acc = np.mean(y_pred == y_true)
                    final_accs.append(acc)
                acc = np.mean(final_accs)
                val_accuracies[seed,j-1] = acc

                unlabeled_pred = lp.iter_sp(labels,
                                     graph,
                                     is_labeled,
                                     10 * j,
                                     unlabeled_indices)

                y_pred = np.argmax(unlabeled_pred,axis=1)
                y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
                acc = np.mean(y_pred == y_true)    
                iter_accuracies[seed,j-1] = acc
            lp = LP()
            unlabeled_pred = lp.closed_sp(labels,
                 graph,
                 labeled_indices,
                 unlabeled_indices)

            y_pred = np.argmax(unlabeled_pred,axis=1)
            y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
            acc = np.mean(y_pred == y_true)    
            closed_accuracies[seed] = acc
            

flip_cora


In [11]:
np.savetxt('t_lp_closed.csv',closed_accuracies)

In [12]:
np.savetxt('t_lp_iter.csv',iter_accuracies)

In [13]:
np.savetxt('t_lp_val.csv',val_accuracies)

In [9]:
val_accuracies

array([[ 0.52,  0.36,  0.4 ,  0.4 ,  0.4 ,  0.36,  0.32,  0.32,  0.28,
         0.28],
       [ 0.6 ,  0.6 ,  0.6 ,  0.6 ,  0.6 ,  0.6 ,  0.6 ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [24]:
np.mean(val_accuracies,axis=1)

array([[ 0.584     ,  0.576     ,  0.576     ,  0.572     ,  0.56      ,
         0.54      ,  0.524     ,  0.516     ,  0.496     ,  0.496     ],
       [ 0.64090909,  0.65436364,  0.65581818,  0.64672727,  0.64127273,
         0.62490909,  0.62709091,  0.61963636,  0.60872727,  0.59381818],
       [ 0.66      ,  0.65875   ,  0.6325    ,  0.60625   ,  0.6025    ,
         0.5875    ,  0.57375   ,  0.57      ,  0.55875   ,  0.55125   ],
       [ 0.48333333,  0.48666667,  0.47666667,  0.47      ,  0.48      ,
         0.48      ,  0.48      ,  0.48      ,  0.48      ,  0.48      ],
       [ 0.105     ,  0.1       ,  0.095     ,  0.10666667,  0.10666667,
         0.10666667,  0.10666667,  0.10666667,  0.10166667,  0.10166667],
       [ 0.399     ,  0.399     ,  0.385     ,  0.394     ,  0.394     ,
         0.398     ,  0.398     ,  0.407     ,  0.412     ,  0.408     ],
       [ 0.54      ,  0.54      ,  0.536     ,  0.54      ,  0.54      ,
         0.524     ,  0.508     ,  0.488     

In [10]:
np.mean(iter_accuracies,axis=1)

array([ 0.37552846,  0.67117886,  0.70565041,  0.65443089,  0.69378049,
        0.61743902,  0.5499187 ,  0.63044715,  0.66886179,  0.49804878])

In [8]:
iter_accuracies

array([[ 0.44796748,  0.42804878,  0.41747967,  0.39674797,  0.38577236,
         0.36829268,  0.34756098,  0.33739837,  0.32520325,  0.30081301],
       [ 0.67682927,  0.68292683,  0.68617886,  0.68658537,  0.68617886,
         0.67886179,  0.66747967,  0.65894309,  0.64715447,  0.64065041],
       [ 0.67601626,  0.7101626 ,  0.71544715,  0.71666667,  0.71585366,
         0.71422764,  0.71056911,  0.70609756,  0.6995935 ,  0.69186992],
       [ 0.59837398,  0.63780488,  0.64674797,  0.65934959,  0.66422764,
         0.66788618,  0.66747967,  0.66747967,  0.66829268,  0.66666667],
       [ 0.67073171,  0.68821138,  0.69593496,  0.70162602,  0.70406504,
         0.70487805,  0.69837398,  0.69471545,  0.69065041,  0.68861789],
       [ 0.6199187 ,  0.62804878,  0.62439024,  0.62276423,  0.62195122,
         0.6199187 ,  0.61707317,  0.61300813,  0.60813008,  0.59918699],
       [ 0.57154472,  0.57642276,  0.57439024,  0.57560976,  0.56544715,
         0.55203252,  0.53739837,  0.52560976

In [44]:
np.mean(val_accuracies,axis=1)

array([[ 0.58333333,  0.575     ,  0.58166667,  0.56      ,  0.54666667,
         0.52      ,  0.51666667,  0.50833333,  0.505     ,  0.50166667],
       [ 0.66484848,  0.66878788,  0.67818182,  0.66636364,  0.65393939,
         0.65333333,  0.64787879,  0.64606061,  0.63969697,  0.62909091],
       [ 0.68      ,  0.66      ,  0.63625   ,  0.61125   ,  0.5975    ,
         0.58875   ,  0.5775    ,  0.56875   ,  0.5625    ,  0.55375   ],
       [ 0.47727273,  0.47272727,  0.46363636,  0.45909091,  0.46818182,
         0.46818182,  0.47727273,  0.47727273,  0.47727273,  0.47727273],
       [ 0.105     ,  0.105     ,  0.105     ,  0.115     ,  0.115     ,
         0.115     ,  0.115     ,  0.115     ,  0.11      ,  0.11      ],
       [ 0.36969697,  0.37878788,  0.36515152,  0.37424242,  0.37575758,
         0.39242424,  0.3969697 ,  0.39242424,  0.40151515,  0.40151515],
       [ 0.56833333,  0.57      ,  0.565     ,  0.56      ,  0.545     ,
         0.53      ,  0.51      ,  0.495     

In [17]:
import pickle as pkl
# pkl.dump(accuracies,open('lp_acc.p','wb')
# pkl.dump(val_accuracies,open('lp_val_acc.p','wb'))

In [18]:
accuracies = pkl.load(open('lp_acc.p','rb'))
val_accuracies = pkl.load(open('lp_val_acc.p','rb'))

In [22]:
val_accuracies.mean(axis=1)

array([[ 0.584     ,  0.576     ,  0.576     ,  0.572     ,  0.56      ,
         0.54      ,  0.524     ,  0.516     ,  0.496     ,  0.496     ],
       [ 0.64090909,  0.65436364,  0.65581818,  0.64672727,  0.64127273,
         0.62490909,  0.62709091,  0.61963636,  0.60872727,  0.59381818],
       [ 0.66      ,  0.65875   ,  0.6325    ,  0.60625   ,  0.6025    ,
         0.5875    ,  0.57375   ,  0.57      ,  0.55875   ,  0.55125   ],
       [ 0.48333333,  0.48666667,  0.47666667,  0.47      ,  0.48      ,
         0.48      ,  0.48      ,  0.48      ,  0.48      ,  0.48      ],
       [ 0.105     ,  0.1       ,  0.095     ,  0.10666667,  0.10666667,
         0.10666667,  0.10666667,  0.10666667,  0.10166667,  0.10166667],
       [ 0.399     ,  0.399     ,  0.385     ,  0.394     ,  0.394     ,
         0.398     ,  0.398     ,  0.407     ,  0.412     ,  0.408     ],
       [ 0.54      ,  0.54      ,  0.536     ,  0.54      ,  0.54      ,
         0.524     ,  0.508     ,  0.488     

In [23]:
accuracies.mean(axis=1)

array([[ 0.61418699,  0.62443089,  0.62264228,  0.61955285,  0.61479675,
         0.60845528,  0.6004065 ,  0.59402439,  0.58747967,  0.57930894],
       [ 0.70087452,  0.7156654 ,  0.71644487,  0.70980989,  0.70391635,
         0.69880228,  0.6938403 ,  0.68813688,  0.68311787,  0.67855513],
       [ 0.66822963,  0.65881384,  0.633101  ,  0.60931441,  0.59114181,
         0.57906476,  0.56820428,  0.55885186,  0.5497909 ,  0.54195919],
       [ 0.55072072,  0.53333333,  0.52657658,  0.52198198,  0.52081081,
         0.51963964,  0.51873874,  0.51783784,  0.51774775,  0.51711712],
       [ 0.2130946 ,  0.21042223,  0.20737573,  0.20897916,  0.20994121,
         0.21084981,  0.21074292,  0.21052913,  0.20983431,  0.20935329],
       [ 0.52725096,  0.5362069 ,  0.53922414,  0.54300766,  0.54310345,
         0.5427682 ,  0.5414272 ,  0.54195402,  0.54171456,  0.5414751 ],
       [ 0.62995935,  0.6402439 ,  0.64308943,  0.63776423,  0.62930894,
         0.61837398,  0.60699187,  0.59817073